## Initialization Script
This cell contains the initialization script for setting up the Flask application with SQLAlchemy and Flask-Migrate.


In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from config import Config
import logging

# Initialize extensions
db = SQLAlchemy()
migrate = Migrate()

def create_app():
    # Create and configure the Flask application
    app = Flask(__name__)
    app.config.from_object(Config)
    
    # Initialize extensions
    db.init_app(app)
    migrate.init_app(app, db)
    
    # Set up logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    # Import and register blueprints
    from . import routes, models
    
    # Example of how to register a blueprint
    # from .routes import main as main_blueprint
    # app.register_blueprint(main_blueprint)
    
    @app.errorhandler(500)
    def internal_error(error):
        db.session.rollback()
        return "500 error"
    
    @app.errorhandler(404)
    def not_found_error(error):
        return "404 error"
    
    return app



## Enhanced OpenAI Integration Script
This cell contains the enhanced script for integrating with the OpenAI API, with added flexibility for model selection, adjustable token limits, and improved error handling.


In [ ]:
import openai
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set the OpenAI API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

def ask_openai(prompt, model="text-davinci-003", max_tokens=150):
    """
    Send a prompt to the OpenAI API and return the generated response.

    Args:
        prompt (str): The input prompt to send to the OpenAI model.
        model (str, optional): The OpenAI model to use. Defaults to "text-davinci-003".
        max_tokens (int, optional): The maximum number of tokens to generate. Defaults to 150.

    Returns:
        str: The text generated by the OpenAI model in response to the prompt.
    """
    try:
        response = openai.Completion.create(
            engine=model,
            prompt=prompt,
            max_tokens=max_tokens
        )
        return response.choices[0].text.strip()
    except openai.error.OpenAIError as e:
        logging.error(f"OpenAI API error: {e}")
        return None
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return None

# Example usage
if __name__ == "__main__":
    prompt = "Write a short story about a cat who can talk."
    response = ask_openai(prompt, model="text-davinci-003", max_tokens=200)
    if response:
        print(response)


## Combined OCR Module
This cell contains the combined OCR module that uses Google Vision for OCR and OpenAI for text enhancement. It includes improved logging, error handling, and flexible configuration.


In [ ]:
import logging
from google_vision_ocr import perform_ocr, set_credentials
from openai_post_processing import enhance_ocr_text

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def process_image(image_path, credentials_path=None):
    """
    Process an image to extract and enhance text using Google Vision OCR and OpenAI.

    Args:
        image_path (str): The path to the image file.
        credentials_path (str, optional): The path to the Google Cloud credentials JSON file.

    Returns:
        list: A list of enhanced texts extracted from the image.
    """
    try:
        if credentials_path:
            set_credentials(credentials_path)
            logging.info(f"Credentials set from: {credentials_path}")
        ocr_texts = perform_ocr(image_path)
        logging.info(f"OCR completed successfully for image: {image_path}")
        enhanced_texts = [enhance_ocr_text(text.description) for text in ocr_texts]
        logging.info("Text enhancement completed successfully.")
        return enhanced_texts
    except FileNotFoundError:
        logging.error(f"Image file not found: {image_path}")
        return []
    except ValueError as e:
        logging.error(f"Error during OCR or text enhancement: {e}")
        return []
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return []

# Ensure example usage only runs when the script is executed directly
if __name__ == "__main__":
    image_path = input("Enter the path to your image: ")
    credentials_path = input("Enter the path to your Google Cloud credentials JSON file (optional): ").strip()
    try:
        results = process_image(image_path, credentials_path if credentials_path else None)
        if results:
            print("\nEnhanced Texts:")
            for result in results:
                print(result)
        else:
            print("No texts were enhanced.")
    except Exception as e:
        logging.error(f"An error occurred: {e}")
